<a href="https://colab.research.google.com/github/GrunCrow/MUICE_UCO_Code/blob/main/1%C2%BA%20Cuatrimestre/%5BBDA%5D%20-%20Introducci%C3%B3n%20al%20Big%20Data%20An%C3%A1lisis/Pr%C3%A1cticas/Bloque%202/02_Map_Filter_Reduce_en_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b>`map`, `filter`, `reduce` en Python</b>
## <i>Big Data Analytics</i>

Curso 2023/24

Prof. *Dr. José Raúl Romero Salguero*

---



Para entender el modelo de desarrollo con MapReduce en Python, es necesario conocer primero los fundamentos de la programación funcional en este lenguaje, así como las funciones `map`, `filter` y `reduce` de Python. Este es el tema que se aborda en este *notebook*.

---

# **Expresiones lambda en Python**

Para entender cómo idear un código de MapReduce, tomamos como referencia el paradigma de la **programación funcional**, en el que las funciones son *ciudadanos de primer orden*, donde sus expresiones pueden ser asignadas a variables que se pasan como parámetros en cualquier función. Para ello, hay primero que entender las **expresiones lambda**.


## ¿Qué son las expresiones lambda?
Las **expresiones lambda** nos permiten centrarnos en *qué se hace*, no tanto en el estado o el *cómo se hace*.

En general, una expresión lambda es una función sin nombre y una única línea de código, de la forma: `lamdba arg(s): expr`.

Su función es evaluar la expresión `expr` para un(os) argumento(s) `arg(s)` dado(s).

##¿Por qué se utilizan?
* En expresiones lógicas, hace el código más legible y fácil de comprender.
* Especialmente adecuado para códigos de un solo uso.

##¿Cuándo **no** utilizarlas?
* Cuando se necesite ejecutar múltiples expresiones.
* En operaciones que necesiten condicionales o expresiones anidadas:
> *si no se puede leer y entender rápidamente, no es una expresión lambda adecuada* <br> → utiliza una función

##¿Qué relación tienen con nuestra asignatura?
En Python, se utilizan funciones lambda como argumento de `map()` y `filter()`, denominadas **funciones de orden superior** (*higher-order functions*).

> En contraposición a las **funciones de primer orden**, una de orden superior toma una función como argumento o devuelve una función




**Ejemplo**: Queremos multiplicar por 2 los elementos de una lista.

In [1]:
def duplicar_en(data):
    res = []
    for i in data:
        res.append(2 * i)
    return res

**Ejemplo**: Queremos multiplicar por 6 los elementos de una lista.

In [2]:
def sextuplicar_en(data):
    res = []
    for i in data:
        res.append(6 * i)
    return res

Ya tenemos dos operaciones, `duplicar_en` y `sextuplicar_en`, que utilizaremos según lo que deseemos hacer con los valores de la lista.

In [3]:
sextuplicar_en([1,2,4,8,16,32,64,128])

[6, 12, 24, 48, 96, 192, 384, 768]

In [4]:
duplicar_en([1,2,4,8,16,32,64,128])

[2, 4, 8, 16, 32, 64, 128, 256]

**¡Estamos duplicando código!**

¿Cómo podemos evitar duplicar código?

In [5]:
def multiplicar_en(n, data):
    res = []
    for i in data:
        res.append(n * i)
    return res

Ahora el código anterior ya no se duplica:

In [6]:
# Sextuplicar
multiplicar_en(6,[1,2,4,8,16,32,64,128])
# Duplicar
multiplicar_en(2,[1,2,4,8,16,32,64,128])

[2, 4, 8, 16, 32, 64, 128, 256]

¿Solucionado?

**Ejemplo**: Supongamos que queremos elevar al cuadrado los elementos de una lista.

In [7]:
def cuadrado_en(data):
    res = []
    for i in data:
        res.append(i * i)
    return res

In [8]:
import random
cuadrado_en([random.randint(0,1000) for _ in range(10)])

[96100, 755161, 25921, 956484, 288369, 40804, 946729, 390625, 33856, 213444]

Al código anterior no puedo aplicarle un operando (desconocido), por lo que seguiremos duplicando el código para implementar el recorrido por la lista para realizar la operación.

Consideramos entonces el siguiente código.

In [9]:
def cuadrado(x):
  return x*x

def doble(x):
  return 2*x

#Podemos redefinir las funciones de antes:
def cuadrado_en(data):
    res = []
    for i in data:
        res.append(cuadrado(i))
    return res

def duplicar_en(data):
    res = []
    for i in data:
        res.append(doble(i))
    return res

duplicar_en([1,2,4,8,16,32,64,128])

[2, 4, 8, 16, 32, 64, 128, 256]

In [10]:
cuadrado_en([1,2,4,8,16,32,64,128])

[1, 4, 16, 64, 256, 1024, 4096, 16384]

Incluso refactorizando el código, se mantiene la duplicidad.

**¿Puede evitarse esta duplicidad de código?**

## Funciones como valores

Dado que todo el cuerpo del comportamiento es similar, cambiando la expresión que se aplica, podemos pasar esta **expresión como valor a la función** que realiza la operación en el recorrido de la lista. Se trataría de una *inyección de dependencias*.

Ya tenemos las expresiones definidas antes, por lo que ahora definimos el comportamiento de cómo se realiza la operación sobre los elementos de la lista.

In [11]:
def aplicar_f_en(f, data):
    res = []
    for i in data:
        res.append(f(i))
    return res

Realizamos de nuevo las operaciones `cuadrado` y `multiplicar`.

In [12]:
aplicar_f_en(cuadrado, [1,2,4,8,16,32,64,128])

[1, 4, 16, 64, 256, 1024, 4096, 16384]

In [13]:
aplicar_f_en(doble, [1,2,4,8,16,32,64,128])

[2, 4, 8, 16, 32, 64, 128, 256]

Pero mantenemos una limitación: **¿hay necesidad de dar un nombre** (de función) **a una expresión sencilla?**

Eliminemos las funciones `doble` y `cuadrado`, ya que *conocemos sus expresiones*, *no requieren más parámetros* y *son limitantes* respecto a qué operaciones artiméticas se pueden realizar sobre los elementos de la lista.

## <b>Funciones lambda</b>



Las expresiones lambda ahorran tiempo y código.

Volvamos a las operaciones `cuadrado` y `duplicar`:

In [14]:
aplicar_f_en(lambda x:x*x, [1,2,4,8,16,32,64,128])

[1, 4, 16, 64, 256, 1024, 4096, 16384]

In [15]:
aplicar_f_en(lambda x:2*x, [1,2,4,8,16,32,64,128])

[2, 4, 8, 16, 32, 64, 128, 256]

# **`map`** y **`reduce`** en Python

Las funciones `map`, `filter` y `reduce` ayudan a la manipulación de colecciones en Python. Python dispone de una implementación nativa de estas funciones, siendo el **núcleo de tecnologías como Spar**k, frameworks para la manipulación y almacenamiento de datos.

Independientemente de si se utilizan en Spark o no, resultan de gran ayuda también para desarrollos en lenguaje Python nativo.

## Función `map`

La función `map` tiene la siguiente estructura:
> `map(fn, col)`

Esta función transforma cada elemento de un iterable o colección `col` pasándolo por la función `fn` y produce una nueva colección __con el mismo número de elementos__. El objeto creado es de tipo `map`, por lo que puede ser necesario su transformación.

Una de las ventajas notorias de `map` es que su implementación **es más rápida que el uso de bucles sobre colecciones**, por lo que es una forma más adecuada de iteración cuando se cumplen las condiciones para ello.  

In [16]:
list(map(lambda x: x*x, [1, 2, 3, 4, 5]))

[1, 4, 9, 16, 25]

Observa que `map` no necesariamente requiere una función con nombre, por lo que el **uso de lambda** puede mejorar la comprensibilidad y tiempos de la llamada.

La correspondencia entre el iterable de entrada y el de salida es siempre exacta en cuanto a números de elementos: **función N:N**.

Ejemplo: Conteo de letras de los elementos de una lista.

In [17]:
# El uso de len(x) para el conteo de caracteres de una cadena es válido en Python3
map_long = map(lambda x:len(x), ['Anakin', 'Luke', 'Han', 'Leia', 'Obi-Wan', 'Yoda', 'Kylo', 'Chewbacca'])
list(map_long)

[6, 4, 3, 4, 7, 4, 4, 9]

La función `map` desarrollada en la celda de arriba es la optimización apropiada con uso de lambda para la formulación. Una versión equivalente pero no tan correcta sería:

In [18]:
def contar_letras(x):
  return len(x)

list(map(contar_letras, ['Anakin', 'Luke', 'Han', 'Leia', 'Obi-Wan', 'Yoda', 'Kylo', 'Chewbacca']))

[6, 4, 3, 4, 7, 4, 4, 9]

## Función `filter`

La función filter tiene la siguiente estructura:

> `filter(fn, col)`

Esta función crea una nueva lista a partir de los elementos del iterable `col` si satisfacen la condición establecida en `fn`, esto es, si `fn` devuelve `True`.

`filter` es equivalente a implementar un bucle con un condicional. Sin embargo, tanto en *frameworks* como en Python nativo resulta una forma mucha más eficiente de realizar esta implementación.

En este caso, la correspondencia entre el iterable de entrada y el de salida no es exacta (como en `map`), sino que será una **relación N:M / M <= N**


**Ejemplo**: Supongamos que solo nos gustan los nombres cortos (4 o menos letras)

In [19]:
def esNombreCorto(x):
  return len(x) < 5

n_cortos = filter(esNombreCorto, ['Anakin', 'Luke', 'Han', 'Leia', 'Obi-Wan', 'Yoda', 'Kylo', 'Chewbacca'])
list(n_cortos)

['Luke', 'Han', 'Leia', 'Yoda', 'Kylo']

De forma similar a `map`, aligerar y optimizar el código con el **uso de expresiones lambda** cuando sea posible.

In [20]:
n_cortos = filter(lambda x: len(x) < 5, ['Anakin', 'Luke', 'Han', 'Leia', 'Obi-Wan', 'Yoda', 'Kylo', 'Chewbacca'])
list(n_cortos)

['Luke', 'Han', 'Leia', 'Yoda', 'Kylo']

## Función `reduce`

La función `reduce` tiene la siguiente estructura:

> `reduce(fn, col[, init])`

Esta función aplica la función `fn` sobre elementos del iterable `col` y **devuelve un único valor resultado** de la computación. Por tanto, reduce establece entre sus entradas y salidas una **relación N:1**.

Para saber cómo funciona, supongamos la multiplicación de los valores de una lista:

In [21]:
import functools

functools.reduce(lambda x,y: x*y, [1,2,3,4,5,6,7,8,9]) #[1,2,4,8,16,32,64,128,256])

362880

*NOTA*: Obsérvese que **`reduce` ya no está implementada de forma nativa** desde la versión Python3, por lo que debe utilizarse en el módulo que la define: `functools`

`init` es el valor inicial resultado con el que se comienza la ejecución de `reduce`, incorporándose antes que el primero (elemento 0) cuando se proporciona.

Para la reducción anterior, la secuencia es como sigue:

1.   `1*2` → 2
2.   `2*3` → 6
3.   `6*4` → 24
4.   `24*5` → 120
5.   ...

En caso de haber especificado `init`, se incluiría un paso 0 (anterior al 1), que hubiera realizado la operación `init*1`


*Ejemplo*: Supongamos que queremos obtener el máximo de una lista

In [22]:
import random

lista = [random.randint(0,1000) for _ in range(10)]
print("La lista de elementos es: ", lista)

functools.reduce(lambda x,y: x if x > y else y, lista)

La lista de elementos es:  [471, 233, 896, 198, 208, 860, 606, 273, 317, 576]


896

`reduce` puede combinarse con funciones de `operator` en Python para hacer el código más legible: https://www.geeksforgeeks.org/operator-functions-in-python-set-1/

In [23]:
from operator import gt
from functools import reduce
reduce(lambda x,y: x if gt(x,y) else y, lista)

896

Como puedes observar, `reduce` solo devuelve el elemento resultado. Si estás interesado en recibir los resultados intermedios, puede utilizarse la función `accumulate` de Python, implementada en el módulo `itertools`.

*NOTA*: Precaución porque (1) se alterna el orden de los parámetros respecto a `reduce`; y (2) no se garantiza el mismo rendimiento.

In [24]:
from itertools import accumulate
lista = [random.randint(0,10) for _ in range(10)]
print("Lista de elementos a sumar: ", lista)
print("El resultado de la suma de la lista es: ", reduce(lambda x,y: x+y, lista))
print("Los valores intermedios de la suma son: ", list(accumulate(lista, lambda x,y: x+y)))

Lista de elementos a sumar:  [10, 2, 7, 2, 4, 3, 8, 2, 2, 2]
El resultado de la suma de la lista es:  42
Los valores intermedios de la suma son:  [10, 12, 19, 21, 25, 28, 36, 38, 40, 42]


El uso de las tres funciones, `map`, `filter` y `reduce`, combinadas resulta uno de los conceptos más potentes de Python, y aprovecha al máximo las capacidades de la programación funcional.

*Ejemplo*: Supongamos que tenemos un dataset con datos de las ciudades más populosas de los distintos países, sean las capitales o no. Queremos obtener una única cadena con el listado (separado por comas) de aquellas capitales que tienen más de 10M de población. *Fuente*: (Luck, 2020)

In [25]:
from functools import reduce

# Se define el dataset con algunos datos, incluyendo una columna no realmente necesaria
data=[['Tokyo', 35676000.0, 'primary'], ['New York', 19354922.0, 'nan'], ['Mexico City', 19028000.0, 'primary'], ['Mumbai', 18978000.0, 'admin'], ['São Paulo', 18845000.0, 'admin'], ['Delhi', 15926000.0, 'admin'], ['Shanghai', 14987000.0, 'admin'], ['Kolkata', 14787000.0, 'admin'], ['Los Angeles', 12815475.0, 'nan'], ['Dhaka', 12797394.0, 'primary'], ['Buenos Aires', 12795000.0, 'primary'], ['Karachi', 12130000.0, 'admin'], ['Cairo', 11893000.0, 'primary'], ['Rio de Janeiro', 11748000.0, 'admin'], ['Ōsaka', 11294000.0, 'admin'], ['Beijing', 11106000.0, 'primary'], ['Manila', 11100000.0, 'primary'], ['Moscow', 10452000.0, 'primary'], ['Istanbul', 10061000.0, 'admin'], ['Paris', 9904000.0, 'primary']]

# Primero, se filtran aquellas ciudades que son capitales de país y tienen más de 10M de habitantes
map_obj = filter(lambda x: x[2]=='primary' and x[1]>10000000,data)
# Segundo, se crea una nueva lista a partir del elemento 0 de la lista anterior (los nombres)
map_obj = map(lambda x: x[0], map_obj)
# Tercero, se reduce la lista a una cadena (iniciada con "Capitales:") mediante la concatenación los elementos de la lista anterior (separados por comas)
map_obj = reduce(lambda x,y: x+", "+y, map_obj, 'Capitales:')
print(map_obj)

Capitales:, Tokyo, Mexico City, Dhaka, Buenos Aires, Cairo, Beijing, Manila, Moscow


## Programación funcional y paralelismo

*Fuente*: Phelps, 2016.

La función de `map` se puede paralelizar a nivel de datos fácilmente. Para ello, es <u>importante</u> que la función argumento de `map` esté libre de efectos secundarios (*side effects*). Por ejemplo, se debe evitar trabajar con [datos mutables](https://towardsdatascience.com/https-towardsdatascience-com-python-basics-mutable-vs-immutable-objects-829a0cb1530a#:~:text=An%20object's%20type%20defines%20the%20possible%20values%20and%20operations.,are%20created%20are%20called%20immutable.).  

Un esquema habitual de paralelismo para este tipo de computación es como sigue, que sirve a modo de **plantilla para nuestro código**:

In [26]:
def hacer(fn, res, data, i):
    print("Calculando el ", i, "-ésimo resultado...")
    # Esta operación se planifica en distintos CPU o hilos
    res[i] = fn(data[i])

def fn_map(fn, data):
    # Recordemos que map es una función N:N
    res = [None] * len(data)
    for i in range(len(data)):
        hacer(fn, res, data, i)
    # Esperar aquí a que el resto de hilos termine antes de emitir el resultado...
    return res

# Pongamos un ejemplo de función lambda
fn_map(lambda x: x * x, [random.randint(0,100) for _ in range(10)])

Calculando el  0 -ésimo resultado...
Calculando el  1 -ésimo resultado...
Calculando el  2 -ésimo resultado...
Calculando el  3 -ésimo resultado...
Calculando el  4 -ésimo resultado...
Calculando el  5 -ésimo resultado...
Calculando el  6 -ésimo resultado...
Calculando el  7 -ésimo resultado...
Calculando el  8 -ésimo resultado...
Calculando el  9 -ésimo resultado...


[729, 121, 5625, 961, 3481, 1936, 529, 1936, 1681, 529]

Como vemos, la paralelización dentro se aplica en la invocación al cálculo de `map`.

Ahora, una vez tenemos la plantilla de código para resolver el problema, aplicamos *multi-threading* para paralelizar de forma real. Para ello, utilizaremos el módulo `threading`

En Python, un proceso puede tener varios hilos de ejecución simultánea (paralelo). Se recomienda la lectura del tutorial para comprender los fundamentos: https://www.geeksforgeeks.org/multithreading-python-set-1/

Transformamos el ejemplo anterior en un programa multi-hilo:

In [27]:
import threading
import random
from threading import Thread

def hacer_hilo(fn, res, data, hilos, i):
    # La evaluación de cada función se planifica en un core distinto
    def trabajo():
        print("trabajo(", threading.current_thread().name, "): Procesando datos:", data[i])
        # Es realmente en este punto donde se aplica lambda sobre los datos concretos del hilo
        res[i] = fn(data[i])
        print("trabajo(", threading.current_thread().name, "): Finalizado hilo #", i)
        print("trabajo(", threading.current_thread().name, "): Resultado es", res[i])
    hilos[i] = Thread(target=trabajo)

def fn_map_multihilo(fn, data):
    # El programa principal es el encargado de planificar los trabajos, ejecutar los hilos y esperar a que terminen
    n = len(data)
    res = [None] * n
    hilos = [None] * n
    print("fn_map_multihilo(): Planificando trabajos")
    for i in range(n):
        hacer_hilo(fn, res, data, hilos, i)
    print("fn_map_multihilo(): Iniciando trabajos")
    for i in range(n):
        hilos[i].start()
    print("fn_map_multihilo(): Esperando terminación de hilos")
    for i in range(n):
        hilos[i].join()
    print("¡Terminado!")
    return res

# Invocamos a la función lambda multi-hilo
fn_map_multihilo(lambda x: x * x, [random.randint(0,100) for _ in range(10)])

fn_map_multihilo(): Planificando trabajos
fn_map_multihilo(): Iniciando trabajos
trabajo( Thread-10 (trabajo)trabajo( Thread-11 (trabajo) ): Procesando datos: 48
trabajo( Thread-11 (trabajo) ): Finalizado hilo # 1
trabajo( Thread-11 (trabajo) ): Resultado es 2304
 ): Procesando datos: 45
trabajo( Thread-10 (trabajo) ): Finalizado hilo # 0
trabajo( Thread-10 (trabajo) ): Resultado es 2025
trabajo( Thread-12 (trabajo) ): Procesando datos: 28
trabajo( Thread-12 (trabajo) ): Finalizado hilo # 2
trabajo( Thread-12 (trabajo) ): Resultado es 784
trabajo( Thread-13 (trabajo) ): Procesando datos: 51
trabajo( Thread-14 (trabajo) ): Procesando datos: 92
trabajo( Thread-14 (trabajo) ): Finalizado hilo #trabajo( Thread-13 (trabajo) ): Finalizado hilo # 3
trabajo( Thread-13 (trabajo)  4
trabajo( Thread-14 (trabajo) ): Resultado es 8464
): Resultado es 2601
trabajo( Thread-15 (trabajo) ): Procesando datos: 50
trabajo( Thread-15 (trabajo) ): Finalizado hilo # 5
trabajo( Thread-15 (trabajo) ): Resultad

[2025, 2304, 784, 2601, 8464, 2500, 6724, 121, 961, 8649]

Como observamos en el código anterior, la funcion `fn_map_multihilo` es en realidad el código controlador del paralelismo.

Para **crear un hilo nuevo**, se crea un objeto de `Thread` indicando la función que ejecuta el hilo, `target=trabajo`, y los argumentos que se pasan a la función, `args=(0,)`, que en este caso no se aplica.

Una vez creado el hilo, se debe **inicializar el hilo** con el método `start`. De esta forma, se estarán ejecutando simultáneamente los hilos ya iniciados y el programa principal, `fn_map_multihilo`.

En caso de que cada hilo tuviera una carga computacional diferente, podrían acabar en tiempos distintos y no necesariamente en el mismo orden que se iniciaron. Para **esperar la finalización de un hilo**, esto es, detener la ejecución del programa actual hasta que el hilo acaba, se utiliza el método `join`.

En el caso del ejemplo de `map` mostrado arriba, los hilos se van creando en *cores* separados. Conforme se van inicializando nuevos hilos, los anteriores siguen su ejecución. Cuando todos son creados, ya habrán estado ejecutándose gran parte de los hilos sobre sus respectivas particiones/colecciones de datos. Por tanto, es momento de comenzar a esperar que los hilos acaben.

*NOTA*: Este *notebook* está orientado a conocer el funcionamiento de `map`, `filter`, `reduce`, y formular sus versiones en paralelo más sencillas. No se han considerado otros aspectos (bloqueos y zonas críticas, sincronización, desbalanceo de particiones de datos, etc.) que podrían resultar de interés.

# <b>Referencias</b>

Información adicional para consulta:

* S. Luck, [*Map, Filter and Reduce in Pure Python*](https://towardsdatascience.com/accelerate-your-python-list-handling-with-map-filter-and-reduce-d70941b19e52), 2020
* V. Yordanov, [*Python Basics: Mutable vs Immutable Objects*](https://towardsdatascience.com/https-towardsdatascience-com-python-basics-mutable-vs-immutable-objects-829a0cb1530a#:~:text=Some%20of%20the%20mutable%20data,string%2C%20tuple%2C%20and%20range.), 2019
* S. Phelps, [*Data science and big data with Python*](https://github.com/phelps-sg/python-bigdata), 2016
* Tutorial básico/guía de referencia de Python: https://www.w3schools.com/python/
* Tutorial de Python: https://www.geeksforgeeks.org/python-programming-language/